In [1]:
using Random
using PyPlot

In [7]:
# parameters
nd = 2 # dimensions
nx = 1000

ntyear = 100 # 100-day growing-burning season
ntmature = ntyear * 10   # 10 years to maturity
ntignite = ntyear * 100 # lightning ignites each grid every 100 years
p_grow = 1/ntmature
p_ignite = 1/ntignite

0.0001

In [ ]:
# landscape functions

function interior(A)
    sz = size(A)
    nd = length(sz)
    CartesianIndices(ntuple(d -> 2:sz(d)-1, nd)) # interior indices, not used in periodic
end

"grow mature trees in empty cells with probability p"
function grow!(tree, sincetree, p=p_grow)
    # Loop through each CartesianIndex and grow trees in bare cells
    for I in eachindex(tree)
        # Access or update the array using the CartesianIndex
        if !tree[I]
            if rand() < p
                tree[I] = true
                sincetree[I] = 0 # zero step counter for tree
            end
        end
    end
end

"ignite cells that have trees with some probability"
function ignite!(fire, tree, sincefire, p=p_ignite)
    for I in findall(tree) # .&&.! fire
        # Access or update the array using the CartesianIndex
        if rand() < p
            fire[I] = true
            tree[I] = false
            sincefire[I] = 0 # zero step counter for fire
        end
    end
end

# burn functions
"CartesianIndex to shift s(=1) index in dimension dim"
offset(dim, nd=nd, s=1) = CartesianIndex(ntuple(j -> j==dim ? s : 0, nd))

"Periodic modifying shift index of dimension dim by s(=1)"
function shift(idx::CartesianIndex, offset::CartesianIndex, sz::NTuple{N, Int}) where N
    new_coord = ntuple(d -> mod1(idx[d] + offset[d], sz[d]), N) # N is number of dimensions
    CartesianIndex(new_coord)
end

"propagate fires to neighboring trees on a periodic domain"
function burn!(fire, tree, sincefire)
    sz = size(tree)
    nd = length(sz)
    for I in findall(fire) # all burning cells on the grid
        for D in 1:nd # spread forward and backward in each dimension
            for ofs in offset.(D, nd, [-1, 1])
                target = shift(I, ofs, sz)
                if tree[target] # burn cells if treed
                    fire[target] = true
                    
                    sincefire[target] = 0 # zero step counter for fire
                end
            end
        end
        fire[I] = false # original fires burn out
    end
end

"daily step"
function step!(fire, tree, sincefire, sincetree)
    grow!(tree, sincetree)

    # fires burn across the domain in finite time
    ignite!(fire, tree, sincefire)
    burn!(fire, tree, sincefire)

    # increment age of all trees and time since fire
    sincetree[findall(tree)] .+= 1
    sincefire .+= 1
end


step!

In [ ]:
# initialize
Random.seed!(42)

tree = falses(fill(nx, nd)...) # initialize array
sz = size(tree)
C = CartesianIndices(tree)
fire = falses(sz)

# diagnostics count days
# continuously incrementing, reset by events
sincetree = zeros(Int64, sz)
sincefire = zeros(Int64, sz)
# lastfireinterval = zeros(Int64, sz)
# lasttreeinterval = zeros(Int64, sz)

# single step
step!(fire, tree, sincefire, sincetree)

100000

In [ ]:
# big run
nyear = 1000
ndays = nyear * ntyear

for i in 1:ndays
    step!(fire, tree, sincefire, sincetree)
end

In [ ]:
cm = PyPlot.cm

clf()
subplot(2,2,1)
imshow(tree+2*fire, vmax=2, cmap=PyPlot.cm.Greys)
subplot(2,2,2)
imshow(sincetree); colorbar()
subplot(2,2,4)
imshow(sincefire); colorbar()
gcf()